In [137]:
from typing import TYPE_CHECKING

if TYPE_CHECKING:
    from snakemake_stub import *


import gzip, json, collections
from typing import Sequence, Mapping, Collection
from Bio import SeqIO
import scipy.sparse as sp
import numpy as np
import pandas as pd
import sys
sys.path.append("scripts")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4262b1bf4bf1ffb403c0eb7a42ad5906_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4506eccf78279d93d0e8a34c035e91c5_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/6bda807e3967eae797c7b1b9eeaee8db_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c2a47d89d1d34e789fdf782557bb7194_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c6c5514ada15b890fb27d1e36371554c_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/d964a294c2d0fef56a434c021026281e_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/e1c932db5cd4271709e54d8028824bc9_/lib/python3.12/site-packages")
def init_reverse_complement():
    TRANSLATION_TABLE = str.maketrans("ACTGactg", "TGACtgac")

    def reverse_complement(sequence: str) -> str:
        """
        >>> reverse_complement("AATC")
        'GATT'
        >>> reverse_complement("CCANT")
        'ANTGG'
        """
        sequence = str(sequence)
        return sequence.translate(TRANSLATION_TABLE)[::-1]

    return reverse_complement


reverse_complement = init_reverse_complement()

In [138]:
ref_database = '/home/miaocj/docker_dir/data/metagenome/part1/ref2.fa'
query_reads = '/home/miaocj/docker_dir/data/metagenome/part1/diff_depth2.fa'

In [139]:
ref_reads_tax_list

['Agaricusbisporus',
 'Eremotheciumcymbalariae',
 'Kazachstaniaafricana',
 'Phaeoacremoniumminimum']

In [162]:
def load_reads(fasta_path: str):
    read_sequences = []
    read_names = []
    read_orientations = []
    read_length = []

    with open(fasta_path, "rt") as handle:  # Open gzipped file in text mode
        for record in SeqIO.parse(handle, "fasta"):
            seq = str(record.seq)
            read_sequences.append(seq)
            read_names.append(record.id)
            read_orientations.append("+")
            read_length.append(len(record))

            # Include reverse complement
            read_sequences.append(reverse_complement(seq))
            read_names.append(record.id)
            read_orientations.append("-")
            read_length.append(len(record))

    return read_names, read_orientations, read_sequences,read_length

read_names, read_orientations, read_sequences, read_length = load_reads(ref_database)


In [ ]:
def build_kmer_index(
    read_sequences: Sequence[str],
    k: int,
    *,
    sample_fraction: float,
    min_multiplicity: int,
    seed: int,
) -> Mapping[str, int]:
    kmer_counter = collections.Counter()
    for seq in read_sequences:
        for p in range(len(seq) - k + 1):
            kmer = seq[p : p + k]
            kmer_counter[kmer] += 1

    kmer_spectrum = collections.Counter(x for x in kmer_counter.values() if x <= 10)
    print(kmer_spectrum)

    rng = np.random.default_rng(seed=seed)
    vocabulary = set(
        x
        for x, count in kmer_counter.items()
        if count >= min_multiplicity and rng.random() <= sample_fraction
    )
    vocabulary |= set(reverse_complement(x) for x in vocabulary)
    kmer_indices = {kmer: i for i, kmer in enumerate(vocabulary)}
    return kmer_indices

sample_fraction=0.05
min_multiplicity=2
seed=562104830
kmer_indices = build_kmer_index(        
        read_sequences=read_sequences,
        k=16,
        sample_fraction=sample_fraction,
        min_multiplicity=min_multiplicity,
        seed=seed)


Counter({1: 6853434, 2: 90134, 3: 25438, 4: 10907, 5: 6236, 6: 3622, 7: 2564, 8: 1742, 9: 1320, 10: 810})


In [163]:
qread_names, qread_orientations, qread_sequence, qread_length  = load_reads(query_reads)

In [149]:
ref_reads_tax_list = []
with open(ref_database) as file:
    for lines in file:
        if lines[0] == '>':
            line = lines.strip().split(' ')
            ref_reads_tax_list.append(line[1]+line[2])
ref_read_tax = {i:tax for i,tax in enumerate(ref_reads_tax_list)}
ref_length = {i:read_length[i] for i,tax in enumerate(ref_reads_tax_list)}

flag = 0
que_read_tax = {}
indice_length = {}
with open(query_reads) as file:
    for lines in file:
        if lines[0] == '>':
            end = lines.index('_')
            ref_num = lines[1:end]
            que_read_tax[flag] = ref_num
            indice_length[flag] = qread_length[flag]
            flag +=1

In [151]:
def build_feature_matrix(
    read_sequences: Sequence[str],
    kmer_indices: Mapping[str, int],
    k: int,
) -> tuple[sp.csr_matrix, Sequence[Sequence[int]]]:
    row_ind, col_ind, data = [], [], []
    read_features = []
    for i, seq in enumerate(read_sequences):
        features_i = []
        for p in range(len(seq) - k + 1):
            kmer = seq[p : p + k]
            j = kmer_indices.get(kmer)
            if j is None:
                continue
            features_i.append(j)

        read_features.append(features_i)

        kmer_counts = collections.Counter(features_i)
        for j, count in kmer_counts.items():
            row_ind.append(i)
            col_ind.append(j)
            data.append(count)

    feature_matrix = sp.csr_matrix(
        (data, (row_ind, col_ind)), shape=(len(read_sequences), len(kmer_indices))
    )
    return feature_matrix, read_features

In [126]:
##calculate sensitivity and precision

from sklearn.metrics import precision_score, recall_score  

def evaluate(indices):
    actual = []
    prediction = []
    for query_read_num,x in enumerate(indices):
        neighbor = x[0]
        neighbor = (neighbor-1)/2  if neighbor %2 !=0 else neighbor/2
        query_read_num = (query_read_num-1)/2  if query_read_num %2 !=0 else query_read_num/2
        prediction.append(ref_read_tax[neighbor])
        actual.append(que_read_tax[query_read_num])

    precision = precision_score(actual, prediction,average='macro')
    sensitivity = recall_score(actual, prediction,average='macro')
    ##计算每个类别的
    precision_sep = precision_score(actual, prediction, average=None)  
    sensitivity_sep = recall_score(actual, prediction, average=None)
    return precision,sensitivity,precision_sep,sensitivity_sep

In [127]:
ref_feature_matrix,ref_read_features = build_feature_matrix(read_sequences=read_sequences,kmer_indices=kmer_indices,k=16)

In [128]:
ref_feature_matrix.shape

(8, 14078)

In [129]:
que_feature_matrix,que_read_features = build_feature_matrix(read_sequences=qread_sequences,kmer_indices=kmer_indices,k=16)

In [130]:
que_feature_matrix.shape

(4302, 14078)

In [131]:
sys.path.append("scripts")
sys.path.append("../../scripts")
merged_matrix = sp.vstack([ref_feature_matrix, que_feature_matrix])  
from dim_reduction import scBiMapEmbedding
dim500 = scBiMapEmbedding().transform(merged_matrix,n_dimensions=500)

In [132]:
dim500_ref = dim500[:8]
dim500_que = dim500[9:]

In [133]:
## 降维+精确求解
from sklearn.neighbors import NearestNeighbors  
import numpy as np  

query_vectors = np.array(dim500_que) 
database_vectors = np.array(dim500_ref) 
nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto',metric='cosine').fit(database_vectors)  
distances, indices_exact = nbrs.kneighbors(query_vectors)
precision,sensitivity,precision_sep,sensitivity_sep = evaluate(indices_exact)


In [155]:
actual = []
prediction = []
for query_read_num,x in enumerate(indices_exact):
    neighbor = x[0]
    neighbor = (neighbor-1)/2  if neighbor %2 !=0 else neighbor/2
    query_read_num = (query_read_num-1)/2  if query_read_num %2 !=0 else query_read_num/2
    prediction.append(ref_read_tax[neighbor])
    actual.append(que_read_tax[query_read_num])

In [164]:
for i,tax in ref_read_tax.items():
    pre_indices = [index for index, value in enumerate(prediction) if value == tax]
    pre_len = sum([qread_length[x] for x in pre_indices])
    actual_len = read_length[i]
    pred_depth = pre_len/actual_len
    print(tax)
    print(pred_depth)

Agaricusbisporus
17.496957207641202
Eremotheciumcymbalariae
22.542886990411617
Kazachstaniaafricana
23.512340437859905
Phaeoacremoniumminimum
45.30084371824357


In [159]:
ref_length

{0: 1748887, 1: 897456, 2: 421466, 3: 592725}

In [109]:
target_class = 'Encephalitozoonromaleae'
true_positive = sum([1 for p, a in zip(prediction, actual) if p == target_class and a == target_class])
false_positive = sum([1 for p, a in zip(prediction, actual) if p == target_class and a != target_class])
false_negative = sum([1 for p, a in zip(prediction, actual) if p != target_class and a == target_class])

recall = true_positive / (true_positive + false_negative)
precision = true_positive / (true_positive + false_positive)
precision,true_positive,false_positive,false_negative

(0.22524752475247525, 91, 313, 1)

In [ ]:
import matplotlib.pyplot as plt
import seaborn
plt.figure(figsize=(6, 6), dpi=300) 
plt.scatter(precision_sep,sensitivity_sep, label='Data points', marker='o', c=sensitivity_sep, cmap='rainbow')

plt.gca().spines['right'].set_color('none')  
plt.gca().spines['top'].set_color('none') 
plt.xlabel('precision')
plt.ylabel('sensitivity')
plt.margins(0.3) 

In [114]:
##rough evaluate
right_num = 0
for query_read_num,x in enumerate(indices):
    neighbor = x[0]
    neighbor = (neighbor-1)/2  if neighbor %2 !=0 else neighbor/2
    query_read_num = (query_read_num-1)/2  if query_read_num %2 !=0 else query_read_num/2
    if ref_read_tax[neighbor] == que_read_tax[query_read_num]:
        right_num +=1

print(right_num/len(indices))

0.9584399030599837


In [31]:
from dataclasses import dataclass, field
from functools import lru_cache
from typing import Sequence, Type, Mapping, Iterable, Literal
from warnings import warn
from math import ceil
from scipy.sparse._csr import csr_matrix
import numpy as np
import hnswlib

class HNSW():
    def get_neighbors(
        self,
        ref_data: csr_matrix | np.ndarray,
        que_data: csr_matrix | np.ndarray,
        n_neighbors: int,
        metric: Literal["euclidean", "cosine"] = "euclidean",
        *,
        threads: int | None = None,
        M: int = 16,
        ef_construction: int = 200,
        ef_search: int = 50,
    ) -> np.ndarray:
        if metric == "euclidean":
            space = "l2"
        else:
            space = metric

        # Initialize the HNSW index
        p = hnswlib.Index(space=space, dim=ref_data.shape[1])
        if threads is not None:
            p.set_num_threads(threads)
        p.init_index(max_elements=ref_data.shape[0], ef_construction=ef_construction, M=M)
        ids = np.arange(ref_data.shape[0])
        p.add_items(ref_data, ids)
        p.set_ef(ef_search)
        nbr_indices, _ = p.knn_query(que_data, k=n_neighbors)
        return nbr_indices
nbr_indices = HNSW().get_neighbors(ref_data=dim500_ref,que_data=dim500_que,n_neighbors=1)


In [32]:
##dim500_HNSW

indices = HNSW().get_neighbors(ref_data=dim500_ref,que_data=dim500_que,n_neighbors=1)
precision,sensitivity,precision_sep,sensitivity_sep = evaluate(indices)

In [33]:
precision_sep,sensitivity_sep

(array([0.94196891, 0.22524752, 0.9897541 , 0.81634183]),
 array([0.8015873 , 0.98913043, 0.57913669, 0.96286472]))

In [61]:
from dataclasses import dataclass, field
from functools import lru_cache
import collections
from typing import Sequence, Type, Mapping, Iterable, Literal
from warnings import warn
from math import ceil
from scipy import sparse
from scipy.sparse._csr import csr_matrix
import numpy as np
import mmh3



class LowHash():

    @staticmethod
    def _hash(x: int, seed: int) -> int:
        hash_value = mmh3.hash(str(x), seed=seed)
        return hash_value

    @staticmethod
    def _get_hash_values(data: Iterable[int], repeats: int, seed: int) -> np.ndarray:
        rng = np.random.default_rng(seed)
        hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeats, dtype=np.uint64)
        hash_values = []
        for k in range(repeats):
            s = hash_seeds[k]
            for x in data:
                hash_values.append(LowHash._hash(x, seed=s))
        hash_values = np.array(hash_values, dtype=np.int64)
        return hash_values

    @staticmethod
    def _get_lowhash_count(
        hash_count: int,
        lowhash_fraction: float | None = None,
        lowhash_count: int | None = None,
    ) -> int:
        if lowhash_fraction is None and lowhash_count is None:
            raise TypeError(
                "Either `lowhash_fraction` or `lowhash_count` must be specified."
            )
        if lowhash_fraction is not None and lowhash_count is not None:
            raise TypeError(
                f"`lowhash_fraction` and `lowhash_count` cannot be specified at the same time. {lowhash_fraction=} {lowhash_count=}"
            )

        if lowhash_fraction is not None:
            lowhash_count = ceil(hash_count * lowhash_fraction)
            lowhash_count = max(lowhash_count, 1)
        if lowhash_count is None:
            raise ValueError()
        return lowhash_count
    
    def _lowhash(
        self,
        data: csr_matrix | np.ndarray,
        repeats: int,
        lowhash_fraction: float | None,
        lowhash_count: int | None = None,
        seed: int = 5731343,
        verbose=True,
    ) -> csr_matrix:
        sample_count, feature_count = data.shape
        buckets = sparse.dok_matrix(
            (feature_count * repeats, sample_count), dtype=np.bool_
        )

        # Calculate hash values
        hash_values = self._get_hash_values(
            np.arange(feature_count), repeats=repeats, seed=seed
        )

        # For each sample, find the lowest hash values for its features
        for j in range(sample_count):
            feature_indices = sparse.find(data[j, :] > 0)[1]
            hash_count = feature_indices.shape[0]
            sample_lowhash_count = self._get_lowhash_count(
                hash_count=hash_count,
                lowhash_fraction=lowhash_fraction,
                lowhash_count=lowhash_count,
            )
            for k in range(repeats):
                bucket_indices = feature_indices + (k * feature_count)
                sample_hash_values = hash_values[bucket_indices]
                low_hash_buckets = bucket_indices[
                    np.argsort(sample_hash_values)[:sample_lowhash_count]
                ]
                buckets[low_hash_buckets, j] = 1
            if verbose and j % 1000 == 0:
                print(j, end=" ")
        if verbose:
            print("")
        buckets = sparse.csr_matrix(buckets)
        return buckets

    def _get_adjacency_matrix(
        self,
        data: csr_matrix | np.ndarray,
        buckets: csr_matrix,
        n_neighbors: int,
        min_bucket_size,
        max_bucket_size,
        min_cooccurence_count,
    ) -> np.ndarray:

        # Select neighbor candidates based on cooccurence counts
        row_sums = buckets.sum(axis=1).A1  # type: ignore
        matrix = buckets[
            (row_sums >= min_bucket_size) & (row_sums <= max_bucket_size), :
        ].astype(np.uint8)
        cooccurrence_matrix = matrix.T.dot(matrix)

        neighbor_dict = collections.defaultdict(dict)
        nonzero_indices = list(zip(*cooccurrence_matrix.nonzero()))
        for i, j in nonzero_indices:
            if i >= j: 
                continue

            count = cooccurrence_matrix[i, j]
            neighbor_dict[i][j] = count
            neighbor_dict[j][i] = count

        # Construct neighbor matrix
        n_rows = data.shape[0]
        nbr_matrix = []
        for i in range(n_rows)[8:]:
            row_nbr_dict = {
                j: count
                for j, count in neighbor_dict[i].items()
                if count >= min_cooccurence_count and j < 8
            }
            neighbors = list(
                sorted(row_nbr_dict, key=lambda x: row_nbr_dict[x], reverse=True)
            )[:n_neighbors]
            nbr_matrix.append(neighbors)
        return nbr_matrix

    def get_neighbors(
        self,
        data: csr_matrix | np.ndarray,
        n_neighbors: int,
        lowhash_fraction: float | None = None,
        lowhash_count: int | None = None,
        repeats=100,
        min_bucket_size=2,
        max_bucket_size=float("inf"),
        min_cooccurence_count=1,
        *,
        seed=1,
        verbose=True,
    ) -> np.ndarray:

        buckets = self._lowhash(
            data,
            repeats=repeats,
            lowhash_fraction=lowhash_fraction,
            lowhash_count=lowhash_count,
            seed=seed,
            verbose=verbose,
        )
        nbr_matrix = self._get_adjacency_matrix(
            data,
            buckets,
            n_neighbors=n_neighbors,
            min_bucket_size=min_bucket_size,
            max_bucket_size=max_bucket_size,
            min_cooccurence_count=min_cooccurence_count,
        )
        return nbr_matrix


In [62]:
COVERAGE_DEPTH = 20
max_bucket_size = COVERAGE_DEPTH * 1.5
indices = LowHash().get_neighbors(data=merged_matrix,repeats=100,lowhash_count=20,n_neighbors=1,
            max_bucket_size=max_bucket_size,
            seed=458)

0 1000 2000 3000 


In [67]:
##calculate sensitivity and precision

from sklearn.metrics import precision_score, recall_score  

def evaluate(indices):
    actual = []
    prediction = []
    for query_read_num,x in enumerate(indices):
        if len (x) > 0:
            neighbor = x[0]
            neighbor = (neighbor-1)/2  if neighbor %2 !=0 else neighbor/2
            query_read_num = (query_read_num-1)/2  if query_read_num %2 !=0 else query_read_num/2
            prediction.append(ref_read_tax[neighbor])
            actual.append(que_read_tax[query_read_num])

    precision = precision_score(actual, prediction,average='macro')
    sensitivity = recall_score(actual, prediction,average='macro')
    ##计算每个类别的
    precision_sep = precision_score(actual, prediction, average=None)  
    sensitivity_sep = recall_score(actual, prediction, average=None)
    return precision,sensitivity,precision_sep,sensitivity_sep

In [66]:

empty_count = sum([1 for sublist in indices if len(sublist) == 0])
len(indices)

3192

In [68]:
precision,sensitivity,precision_sep,sensitivity_sep = evaluate(indices)

In [69]:
precision_sep,sensitivity_sep
##输入种类比较少的时候，一些本身reads很少的genome 可以设置一个阈值，超过阈值的才被判断为近邻/

(array([0.98910082, 0.23989218, 0.90131579, 0.89467593]),
 array([0.68815166, 1.        , 0.83664122, 0.99357326]))